In [14]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tweet-sentiment-extraction/sample_submission.csv
/kaggle/input/tweet-sentiment-extraction/train.csv
/kaggle/input/tweet-sentiment-extraction/test.csv


In [15]:
def jaccard(str1, str2):
    a = set(str1.lower().split())
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [16]:
train = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/train.csv')
test = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/test.csv')
subm = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/sample_submission.csv')

In [24]:
train = train.dropna()
train

,textID,text,selected_text,sentiment,pred
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,"I`d have responded, if I were going"
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,sooo sad i
2,088c60f138,my boss is bullying me...,bullying me,negative,boss is bullying
3,9642c003ef,what interview! leave me alone,leave me alone,negative,what interview !
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"sons of ,"
...,...,...,...,...,...
27476,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative,denver husband lost
27477,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative,i ve wondered
27478,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive,yay good for
27479,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive,it was worth


In [23]:
test

,textID,text,sentiment
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative
3,01082688c6,happy bday!,positive
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive
...,...,...,...
3529,e5f0e6ef4b,"its at 3 am, im very tired but i can`t sleep ...",negative
3530,416863ce47,All alone in this old house again. Thanks for...,positive
3531,6332da480c,I know what you mean. My little dog is sinkin...,negative
3532,df1baec676,_sutra what is your next youtube video gonna b...,positive


In [19]:
subm

,textID,selected_text
0,f87dea47db,NaN
1,96d74cb729,NaN
2,eee518ae67,NaN
3,01082688c6,NaN
4,33987a8ee5,NaN
...,...,...
3529,e5f0e6ef4b,NaN
3530,416863ce47,NaN
3531,6332da480c,NaN
3532,df1baec676,NaN


In [ ]:
import re
import pandas as pd
from nltk.corpus import opinion_lexicon
from nltk.tokenize import word_tokenize

# 0. Jaccard функция с защитой от NaN

def jaccard(str1, str2):
    """
    Word-level Jaccard similarity между двумя строками.
    Корректно обрабатывает NaN, преобразуя их в пустые строки.
    """
    s1 = str(str1).lower().split()
    s2 = str(str2).lower().split()
    a = set(s1)
    b = set(s2)
    if not a and not b:
        return 1.0
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

# 1. Загрузка данных
train = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/train.csv')
test = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/test.csv')

# 2. Предобработка текста

def clean_text(text: str) -> str:
    """
    Очищает текст: приводит к нижнему регистру, убирает ссылки и упоминания, лишние пробелы.
    Корректно обрабатывает NaN (преобразуя в пустую строку).
    """
    text = str(text).lower()
    text = re.sub(r'http\S+|www\S+', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'[^a-z0-9\s\.,!\?\'\"-]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# 3. Простая эвристика по лексикону Opinion Lexicon
pos_words = set(opinion_lexicon.positive())
neg_words = set(opinion_lexicon.negative())

def sentiment_score(tokens: list[str]) -> int:
    """
    Считает разницу между количеством положительных и отрицательных слов в списке токенов.
    """
    return sum(1 for w in tokens if w in pos_words) - sum(1 for w in tokens if w in neg_words)


def extract_by_lexicon(text: str, sentiment: str, window_size: int = 3) -> str:
    """
    Для нейтрального твита возвращает весь текст.
    Для остальных выбирает фрагмент длиной {window_size} слов с наибольшим лексическим score.
    Корректно обрабатывает пустой или NaN текст.
    """
    original = str(text) if pd.notna(text) else ''
    cleaned = clean_text(text)
    tokens = word_tokenize(cleaned)
    if sentiment == 'neutral' or len(tokens) <= window_size:
        return original

    best_span = tokens[:window_size]
    best_score = sentiment_score(best_span)
    for i in range(len(tokens) - window_size + 1):
        span = tokens[i:i + window_size]
        sc = sentiment_score(span)
        if (sentiment == 'positive' and sc > best_score) or (sentiment == 'negative' and sc < best_score):
            best_score = sc
            best_span = span

    return ' '.join(best_span)

# 4. Применение к train для проверки качества Jaccard
train['pred'] = train.apply(lambda row: extract_by_lexicon(row.text, row.sentiment), axis=1)
train['jaccard'] = train.apply(lambda r: jaccard(r.selected_text, r.pred), axis=1)
print(f"Mean Jaccard: {train.jaccard.mean():.4f}")

# 5. Пример предсказания на test
subm = test[['textID']].copy()
subm['selected_text'] = test.apply(lambda r: extract_by_lexicon(r.text, r.sentiment), axis=1)
subm.to_csv('submission.csv', index=False)
print("Создан файл submission.csv")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
from lightgbm import LGBMClassifier

# 1. Helper: sliding window span generator (safely handles NaNs)
def generate_spans(text, max_len=5):
    text = str(text)
    tokens = text.split()
    spans = []
    n = len(tokens)
    for length in range(1, min(max_len, n) + 1):
        for start in range(n - length + 1):
            span = " ".join(tokens[start:start + length])
            spans.append(span)
    return list(set(spans))

# 2. Load data
data = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/train.csv')
test = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/test.csv')

# 3. Build training examples
neg_examples = []
for _, row in data.iterrows():
    text = row.text
    selected = row.selected_text
    sentiment = row.sentiment
    if pd.isna(text) or pd.isna(selected):
        continue
    # positive example
    eg = {'span': selected, 'sentiment': sentiment, 'label': 1}
    neg_examples.append(eg)
    # sample negatives
    candidates = generate_spans(text)
    negs = [s for s in candidates if s.lower() != str(selected).lower()]
    if negs:
        sampled = np.random.choice(negs, min(3, len(negs)), replace=False)
        for s in sampled:
            neg_examples.append({'span': s, 'sentiment': sentiment, 'label': 0})
train_df = pd.DataFrame(neg_examples)

# 4. Feature engineering: TF-IDF on span + one-hot sentiment
def get_span(X): return X['span']
def get_sentiment(X): return X[['sentiment']]
span_vect = Pipeline([
    ('extract', FunctionTransformer(get_span, validate=False)),
    ('tfidf', TfidfVectorizer(ngram_range=(1,3), max_features=50000))
])
sent_vect = Pipeline([
    ('extract', FunctionTransformer(get_sentiment, validate=False)),
    ('ohe', OneHotEncoder())
])
features = FeatureUnion([('span', span_vect), ('sentiment', sent_vect)])

# 5. Train/validation split
y = train_df.label
X_train, X_val, y_train, y_val = train_test_split(train_df, y, stratify=y, test_size=0.1, random_state=42)

# 6. Pipeline with LightGBM
pipeline = Pipeline([
    ('features', features),
    ('clf', LGBMClassifier(n_estimators=200, learning_rate=0.1, random_state=42))
])

# 7. Train and evaluate
pipeline.fit(X_train, y_train)
print("Validation accuracy:", pipeline.score(X_val, y_val))

# 8. Inference function

def predict_selected(text, sentiment, max_len=5):
    if sentiment == 'neutral' or pd.isna(text):
        return str(text)
    spans = generate_spans(text, max_len)
    df = pd.DataFrame({'span': spans, 'sentiment': sentiment})
    probs = pipeline.predict_proba(df)[:, 1]
    return spans[np.argmax(probs)]

# 9. Create submission
sub = []
for _, row in test.iterrows():
    pred = predict_selected(row.text, row.sentiment)
    sub.append({'textID': row.textID, 'selected_text': pred})
sub_df = pd.DataFrame(sub)
sub_df.to_csv('submission.csv', index=False)
print("Saved submission_lgbm.csv")

In [34]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from scipy.sparse import hstack

# 1) Загрузка данных
train = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/train.csv')
test  = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/test.csv')

# 2) Подготовка токенов и меток

def prepare_tokens(df):
    rows = []
    for _, r in df.iterrows():
        text, sel = str(r.text), str(r.selected_text)
        toks = text.split()
        sel_toks = sel.split() if sel != 'nan' else []
        start_idx = None
        if sel_toks:
            for i in range(len(toks) - len(sel_toks) + 1):
                if toks[i:i+len(sel_toks)] == sel_toks:
                    start_idx = i
                    break
        end_idx = start_idx + len(sel_toks) - 1 if start_idx is not None else None

        for i, tok in enumerate(toks):
            rows.append({
                'token': tok,
                'pos_idx': i,
                'pos_rel': i / max(1, len(toks)-1),
                'y_start': 1 if i == start_idx else 0,
                'y_end':   1 if i == end_idx   else 0
            })
    return pd.DataFrame(rows)

df_tokens = prepare_tokens(train)

# 3) TF-IDF признаков
tfidf = TfidfVectorizer(analyzer='word', token_pattern=r"(?u)\b\w+\b", lowercase=True)
X_tok_tfidf = tfidf.fit_transform(df_tokens['token'])

# 4) Сбор всех признаков
X_pos = df_tokens[['pos_idx','pos_rel']].values
X = hstack([X_tok_tfidf, X_pos])
y_start = df_tokens['y_start'].values
y_end   = df_tokens['y_end'].values

# 5) Сплит и обучение
X_tr, X_val, ys_tr, ys_val = train_test_split(X, y_start, test_size=0.1, random_state=42)
_, _, ye_tr, ye_val = train_test_split(X, y_end,   test_size=0.1, random_state=42)

clf_start = LogisticRegression(max_iter=200)
clf_end   = LogisticRegression(max_iter=200)
clf_start.fit(X_tr, ys_tr)
clf_end.fit(X_tr, ye_tr)

# 6) Функция предсказания

def predict_selected(text):
    toks = text.split()
    if not toks:
        return ""
    tf = tfidf.transform(toks)
    pos_idx = np.arange(len(toks))[:, None]
    pos_rel = (pos_idx / max(1, len(toks)-1))
    Xnew = hstack([tf, np.hstack([pos_idx, pos_rel])])

    p_start = clf_start.predict_proba(Xnew)[:,1]
    p_end   = clf_end  .predict_proba(Xnew)[:,1]

    best_score = 0
    bi, bj = 0, 0
    for i in range(len(toks)):
        for j in range(i, len(toks)):
            score = p_start[i] * p_end[j]
            if score > best_score:
                best_score, bi, bj = score, i, j

    return ' '.join(toks[bi:bj+1])

# 7) Применение к тесту и сохранение сабмита

def build_submission(test_df, out_path='submission.csv'):
    preds = []
    for _, r in test_df.iterrows():
        pred = predict_selected(str(r.text))
        # Для neutral можно вернуть весь текст, если pred пустой
        if not pred or r.sentiment == 'neutral':
            pred = r.text
        preds.append(pred)

    subm = pd.DataFrame({'textID': test_df.textID, 'selected_text': preds})
    subm.to_csv(out_path, index=False)
    print(f"Saved submission to {out_path}")

# Запуск
build_submission(test)

AttributeError: module 'pandas.io.common' has no attribute 'csv'